In [7]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

#### Data Preparation

In [8]:
#! wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip

In [9]:
#!unzip data.zip

#### Model

- Creation

In [10]:
model = models.Sequential()

inp_shape = (150, 150, 3)

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=inp_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

- Compilation

In [11]:
optimizer = optimizers.SGD(learning_rate=0.002, momentum=0.8)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['acc'])

- Summary

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 175232)            0         
                                                                 
 dense_2 (Dense)             (None, 64)                11214912  
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
__________________________________________

#### Image Datageneeator

In [13]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

- Train generator

In [14]:
train_generator = train_datagen.flow_from_directory('data/train/',
                                                    target_size=(150, 150),
                                                    batch_size=20,
                                                    shuffle=True,
                                                    class_mode='binary')

Found 3677 images belonging to 2 classes.


- Validation Generator

In [15]:
test_generator = test_datagen.flow_from_directory('data/test/',
                                                        target_size=(150, 150),
                                                        batch_size=20,
                                                        shuffle=True,
                                                        class_mode='binary', )

Found 918 images belonging to 2 classes.


- Training the model

In [16]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator)

Epoch 1/10
184/184 [==============================] - 16s 83ms/step - loss: 0.6573 - acc: 0.5907 - val_loss: 0.5969 - val_acc: 0.6841
Epoch 2/10
184/184 [==============================] - 15s 79ms/step - loss: 0.5861 - acc: 0.6875 - val_loss: 0.5484 - val_acc: 0.7113
Epoch 3/10
184/184 [==============================] - 14s 78ms/step - loss: 0.5337 - acc: 0.7427 - val_loss: 0.5367 - val_acc: 0.7516
Epoch 4/10
184/184 [==============================] - 14s 78ms/step - loss: 0.4989 - acc: 0.7669 - val_loss: 0.5481 - val_acc: 0.7092
Epoch 5/10
184/184 [==============================] - 15s 79ms/step - loss: 0.4845 - acc: 0.7789 - val_loss: 0.5193 - val_acc: 0.7669
Epoch 6/10
184/184 [==============================] - 15s 79ms/step - loss: 0.4527 - acc: 0.7974 - val_loss: 0.5193 - val_acc: 0.7462
Epoch 7/10
184/184 [==============================] - 14s 79ms/step - loss: 0.4314 - acc: 0.8115 - val_loss: 0.5281 - val_acc: 0.7397
Epoch 8/10
184/184 [==============================] - 15s 79ms

In [17]:
acc_median = np.median(history.history['acc'])
acc_median

0.7881425023078918

In [18]:
loss_std = np.std(history.history['loss'])
loss_std

0.09230162943143867

#### DATA AUGMENTATION

In [19]:
train_datagen = ImageDataGenerator(
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

In [20]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [21]:
train_generator = train_datagen.flow_from_directory('data/train/',
                                                    target_size=(150, 150),
                                                    batch_size=20,
                                                    shuffle=True,
                                                    class_mode='binary')

Found 3677 images belonging to 2 classes.


In [22]:
validation_generator = test_datagen.flow_from_directory(
    'data/test/',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

Found 918 images belonging to 2 classes.


In [23]:
#@ MODEL TRAINING WITH AUGMENTATION:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator)

Epoch 1/10
  1/184 [..............................] - ETA: 27s - loss: 63.9537 - acc: 0.8000

184/184 [==============================] - 19s 102ms/step - loss: 179502.9531 - acc: 0.5298 - val_loss: 0.6939 - val_acc: 0.5349
Epoch 2/10
184/184 [==============================] - 19s 101ms/step - loss: 0.6930 - acc: 0.5390 - val_loss: 0.6936 - val_acc: 0.5327
Epoch 3/10
184/184 [==============================] - 19s 101ms/step - loss: 0.6905 - acc: 0.5398 - val_loss: 0.6952 - val_acc: 0.5403
Epoch 4/10
184/184 [==============================] - 19s 101ms/step - loss: 0.6952 - acc: 0.5388 - val_loss: 0.6911 - val_acc: 0.5436
Epoch 5/10
184/184 [==============================] - 19s 101ms/step - loss: 0.6920 - acc: 0.5390 - val_loss: 0.6907 - val_acc: 0.5425
Epoch 6/10
184/184 [==============================] - 19s 101ms/step - loss: 0.6912 - acc: 0.5385 - val_loss: 0.6921 - val_acc: 0.5370
Epoch 7/10
184/184 [==============================] - 19s 101ms/step - loss: 0.6893 - acc: 0.5390 - val_loss: 0.6926 - val_acc: 0.5436
Epoch 8/10
184/184 [==============================] - 19s 102

In [24]:
val_acc = history.history['val_acc']

val_loss = history.history['val_loss']

In [25]:
np.mean(val_loss)

0.6931777894496918

In [26]:
val_acc

[0.5348584055900574,
 0.5326797366142273,
 0.5403050184249878,
 0.5435729622840881,
 0.5424836874008179,
 0.5370370149612427,
 0.5435729622840881,
 0.5392156839370728,
 0.5424836874008179,
 0.5381263494491577]

In [27]:
last_5_acc = val_acc[5:10]
last_5_acc

[0.5370370149612427,
 0.5435729622840881,
 0.5392156839370728,
 0.5424836874008179,
 0.5381263494491577]

In [28]:
np.mean(last_5_acc)

0.5400871396064758